# FlowScope Complete Testing Guide

## Overview

FlowScope is an **AI/LLM debugging and observability platform** that helps developers:
- 🔍 **Debug AI chains** - Visualize how prompts flow through LLM pipelines
- 📊 **Monitor performance** - Track response times, token usage, and costs
- 🐛 **Find bottlenecks** - Identify slow or failing steps in AI workflows
- 👥 **Collaborate** - Share debugging sessions with teams

## Testing Priority Order

1. **🌐 Web Interface** - Visual exploration and debugging (START HERE)
2. **🔧 VS Code Extension** - IDE-integrated debugging
3. **📓 Jupyter Integration** - Notebook-based AI development
4. **⚙️ SDK Integration** - Full programmatic integration

## Current SDK Support

✅ **Available Now:**
- JavaScript/TypeScript SDK (Full support)
- Web Interface (React-based)
- VS Code Extension

🚧 **In Development:**
- Python SDK (planned, test file exists)
- Jupyter widgets
- Additional language bindings

# 1. Web Interface SDK Setup

## Quick Start (5 minutes)

The web interface is the **easiest way to start** testing FlowScope. It provides a visual debugging environment for AI applications.

### Prerequisites
- FlowScope backend running on `http://localhost:3001`
- FlowScope frontend running on `http://localhost:5173`

### Testing Steps

In [ ]:
// Test Web Interface Connection
// Run this in browser console at http://localhost:5173

async function testWebInterface() {
    console.log('🔄 Testing FlowScope Web Interface...');
    
    try {
        // 1. Test backend connection
        const healthResponse = await fetch('http://localhost:3001/api/health');
        console.log('✅ Backend health:', await healthResponse.json());
        
        // 2. Test session creation via web API
        const sessionResponse = await fetch('http://localhost:3001/api/sessions', {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                name: 'Web Interface Test Session',
                metadata: { source: 'web-test', timestamp: Date.now() }
            })
        });
        
        const session = await sessionResponse.json();
        console.log('✅ Session created:', session.id);
        
        // 3. Add a test trace
        const traceResponse = await fetch(`http://localhost:3001/api/sessions/${session.id}/traces`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                sessionId: session.id,
                chainId: 'web-test-chain',
                timestamp: Date.now(),
                type: 'prompt',
                status: 'completed',
                data: {
                    prompt: 'Test prompt from web interface',
                    model: 'gpt-4',
                    response: 'Test response'
                }
            })
        });
        
        const trace = await traceResponse.json();
        console.log('✅ Trace added:', trace.id);
        
        console.log(`🎉 Success! View at: http://localhost:5173`);
        
    } catch (error) {
        console.error('❌ Web interface test failed:', error);
    }
}

// Uncomment to run:
// testWebInterface();

# 2. VS Code Extension SDK Configuration

## Installation & Setup

The VS Code extension provides **IDE-integrated debugging** for AI applications directly in your development environment.

### Step 1: Install Extension

In [ ]:
# Install FlowScope VS Code Extension
cd "c:\Users\charlesidu\Downloads\FlowScope\packages\vscode-extension"

# Install the pre-built extension
code --install-extension flowscope-0.1.0.vsix

# OR install from source (for development)
npm install
npm run build
code --install-extension .

# Verify installation
code --list-extensions | grep flowscope

# 3. JavaScript SDK Integration

## Current Status: ✅ **Fully Available**

The JavaScript/TypeScript SDK is the **primary SDK** with full feature support:
- LangChain integration
- LlamaIndex integration  
- Manual tracing
- WebSocket support
- Multiple transport options

### Installation

In [ ]:
// Install FlowScope SDK
// npm install @flowscope/sdk

// Basic JavaScript SDK Usage
import FlowScopeSDK from '@flowscope/sdk';

// Initialize SDK
const flowscope = new FlowScopeSDK({
    projectId: 'my-ai-project',
    apiEndpoint: 'http://localhost:3001',
    debug: true
});

async function testJavaScriptSDK() {
    console.log('🔄 Testing JavaScript SDK...');
    
    try {
        // Initialize the SDK
        await flowscope.init();
        
        // Start a debugging session
        const session = flowscope.startSession('js-sdk-test', {
            user: 'developer',
            environment: 'development'
        });
        
        console.log('✅ Session started:', session.id);
        
        // Manual trace example
        const trace = flowscope.trace('customer-support-chain', {
            type: 'prompt',
            data: {
                input: 'How do I return an item?',
                model: 'gpt-4',
                temperature: 0.7
            },
            metadata: {
                userId: 'user-123',
                source: 'chat-widget'
            }
        });
        
        // Simulate processing time
        await new Promise(resolve => setTimeout(resolve, 1000));
        
        // Complete the trace
        trace.complete({
            output: 'To return an item, please visit our returns page...',
            tokens: { input: 25, output: 45, total: 70 },
            cost: 0.0014
        });
        
        console.log('✅ Trace completed:', trace.id);
        
        // Flush traces to backend
        await flowscope.flush();
        
        console.log('🎉 JavaScript SDK test successful!');
        
    } catch (error) {
        console.error('❌ JavaScript SDK test failed:', error);
    }
}

// Uncomment to run:
// testJavaScriptSDK();

# 4. Python SDK Integration

## Current Status: 🚧 **In Development**

The Python SDK is **planned but not yet implemented**. However, you can still integrate FlowScope with Python applications using direct API calls.

### Available Now: Direct API Integration

In [ ]:
# Python integration via direct API calls
import requests
import time
import json
from datetime import datetime

class FlowScopePython:
    """Temporary Python integration until official SDK is released"""
    
    def __init__(self, api_endpoint="http://localhost:3001"):
        self.api_endpoint = api_endpoint
        self.session_id = None
    
    def create_session(self, name, metadata=None):
        """Create a new FlowScope session"""
        response = requests.post(f"{self.api_endpoint}/api/sessions", json={
            "name": name,
            "metadata": metadata or {}
        })
        response.raise_for_status()
        session = response.json()
        self.session_id = session["id"]
        return session
    
    def add_trace(self, chain_id, trace_type, data, status="completed"):
        """Add a trace to the current session"""
        if not self.session_id:
            raise ValueError("No session created. Call create_session() first.")
        
        trace_data = {
            "sessionId": self.session_id,
            "chainId": chain_id,
            "timestamp": int(time.time() * 1000),
            "type": trace_type,
            "status": status,
            "data": data
        }
        
        response = requests.post(
            f"{self.api_endpoint}/api/sessions/{self.session_id}/traces",
            json=trace_data
        )
        response.raise_for_status()
        return response.json()

# Test Python integration
def test_python_integration():
    print("🔄 Testing Python FlowScope integration...")
    
    try:
        # Initialize FlowScope
        flowscope = FlowScopePython()
        
        # Create session
        session = flowscope.create_session(
            "Python LangChain Test",
            {"language": "python", "framework": "langchain"}
        )
        print(f"✅ Session created: {session['id']}")
        
        # Simulate LangChain workflow
        traces = [
            {
                "chain_id": "langchain-rag-pipeline",
                "type": "prompt",
                "data": {
                    "prompt": "What are the benefits of renewable energy?",
                    "template": "Answer based on context: {context}\\nQuestion: {question}",
                    "model": "gpt-4"
                }
            },
            {
                "chain_id": "langchain-rag-pipeline", 
                "type": "function_call",
                "data": {
                    "function": "vector_search",
                    "query": "renewable energy benefits",
                    "top_k": 5
                }
            },
            {
                "chain_id": "langchain-rag-pipeline",
                "type": "response", 
                "data": {
                    "response": "Renewable energy offers several key benefits...",
                    "sources": ["doc1.pdf", "doc2.pdf"],
                    "confidence": 0.92
                }
            }
        ]
        
        for trace in traces:
            result = flowscope.add_trace(**trace)
            print(f"✅ Added {trace['type']} trace: {result['id']}")
            time.sleep(0.5)  # Simulate processing time
        
        print("🎉 Python integration test successful!")
        print(f"📊 View traces at: http://localhost:5173")
        
    except Exception as error:
        print(f"❌ Python integration test failed: {error}")

# Uncomment to run:
# test_python_integration()

# 5. Jupyter Notebook Integration Examples

## Where Jupyter Fits: **Between VS Code and SDK Integration**

Jupyter integration is perfect for **AI/ML experimentation** where you want to:
- Test LLM prompts interactively
- Debug AI pipelines step-by-step  
- Share reproducible AI debugging sessions
- Analyze trace data with pandas/matplotlib

### Integration Priority: **3rd Place**
1. Web Interface (visual exploration)
2. VS Code Extension (IDE integration)  
3. **🔸 Jupyter Integration** (experimental debugging) ← YOU ARE HERE
4. SDK Integration (production integration)

In [ ]:
# Jupyter Notebook FlowScope Integration
# This notebook cell demonstrates AI debugging in Jupyter

import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython.display import display, HTML

# FlowScope Jupyter Helper Class
class FlowScopeJupyter:
    def __init__(self, api_endpoint="http://localhost:3001"):
        self.api_endpoint = api_endpoint
        self.session_id = None
        
    def start_notebook_session(self, notebook_name):
        """Start a FlowScope session for this notebook"""
        response = requests.post(f"{self.api_endpoint}/api/sessions", json={
            "name": f"Jupyter: {notebook_name}",
            "metadata": {
                "environment": "jupyter",
                "notebook": notebook_name,
                "started_at": time.time()
            }
        })
        self.session_id = response.json()["id"]
        
        # Display session info
        display(HTML(f"""
        <div style="background: #e8f5e8; padding: 10px; border-radius: 5px;">
            <strong>🚀 FlowScope Session Started</strong><br>
            Session ID: <code>{self.session_id}</code><br>
            <a href="http://localhost:5173" target="_blank">📊 View in FlowScope Dashboard</a>
        </div>
        """))
        
        return self.session_id
    
    def trace_ai_step(self, step_name, prompt, response, metadata=None):
        """Trace an AI step directly from Jupyter"""
        trace_data = {
            "sessionId": self.session_id,
            "chainId": "jupyter-ai-experiment",
            "timestamp": int(time.time() * 1000),
            "type": "prompt",
            "status": "completed",
            "data": {
                "step": step_name,
                "prompt": prompt,
                "response": response,
                "metadata": metadata or {}
            }
        }
        
        response = requests.post(
            f"{self.api_endpoint}/api/sessions/{self.session_id}/traces",
            json=trace_data
        )
        
        # Visual feedback in notebook
        print(f"✅ Traced: {step_name}")
        return response.json()
    
    def get_session_analytics(self):
        """Get analytics for the current session"""
        response = requests.get(f"{self.api_endpoint}/api/sessions/{self.session_id}/traces")
        traces = response.json()
        
        # Create simple analytics
        df = pd.DataFrame([{
            'step': trace.get('data', {}).get('step', 'Unknown'),
            'timestamp': trace['timestamp'],
            'type': trace['type']
        } for trace in traces])
        
        return df

# Example: AI Prompt Engineering Experiment
def jupyter_ai_experiment():
    """Example of using FlowScope in AI experimentation"""
    
    # Start FlowScope session
    fs = FlowScopeJupyter()
    fs.start_notebook_session("AI Prompt Engineering Experiment")
    
    # Experiment with different prompts
    prompts = [
        "Explain quantum computing simply",
        "Explain quantum computing like I'm 5",
        "Explain quantum computing with analogies"
    ]
    
    responses = [
        "Quantum computing uses quantum mechanics...",
        "Imagine quantum computers are like magic boxes...", 
        "Quantum computing is like a coin that can be heads and tails..."
    ]
    
    # Trace each experiment
    for i, (prompt, response) in enumerate(zip(prompts, responses)):
        fs.trace_ai_step(
            f"prompt_experiment_{i+1}",
            prompt,
            response,
            {"experiment_type": "prompt_engineering", "iteration": i+1}
        )
    
    # Analyze results
    analytics_df = fs.get_session_analytics()
    display(analytics_df)
    
    print("🎉 Jupyter AI experiment complete!")
    print("📊 View detailed traces in FlowScope Dashboard")

# Uncomment to run the experiment:
# jupyter_ai_experiment()

# 6. Cross-Platform SDK Comparison

## SDK Feature Matrix

| Feature | JavaScript/TypeScript | Python | Jupyter | VS Code Extension |
|---------|----------------------|--------|---------|-------------------|
| **Status** | ✅ Full Support | 🚧 API Integration | 🚧 API Integration | ✅ Available |
| **Auto-tracing** | ✅ LangChain, LlamaIndex | 🔄 Planned | ⚠️ Manual only | ✅ Code analysis |
| **Real-time** | ✅ WebSocket support | ❌ HTTP only | ❌ HTTP only | ✅ Live debugging |
| **Ease of use** | 🟡 Moderate | 🟢 Simple API calls | 🟢 Simple API calls | 🟢 Point & click |
| **Best for** | Production apps | Prototyping | Experimentation | Development |

## Current Recommendations

### Start with: **JavaScript SDK** (Most Complete)
- Full feature support
- Production-ready
- Best documentation
- Framework integrations

### For Python users: **Direct API Integration**
- Simple HTTP requests
- Works today
- No external dependencies
- Easy to understand

### For experimentation: **Jupyter Integration**  
- Interactive debugging
- Visual feedback
- Great for prompt engineering
- Data analysis capabilities

## Implementation Priority

1. **JavaScript SDK** - If you're building Node.js/TypeScript AI apps
2. **Direct API calls** - If you're using Python/other languages  
3. **VS Code Extension** - For development workflow integration
4. **Jupyter integration** - For research and experimentation

# Testing Checklist & Next Steps

## Quick Testing Checklist

### ✅ Phase 1: Web Interface (5 minutes)
- [ ] Open http://localhost:5173
- [ ] Run cell 2 (JavaScript web test) in browser console
- [ ] Create a new session manually
- [ ] Explore Chain View, Timeline, and Node Details

### ✅ Phase 2: VS Code Extension (10 minutes)  
- [ ] Install extension using cell 4 commands
- [ ] Open a Python/JavaScript file with AI code
- [ ] Test FlowScope panel integration
- [ ] Try debugging features

### ✅ Phase 3: Jupyter Integration (15 minutes)
- [ ] Run cell 8 (Jupyter AI experiment)
- [ ] Test interactive tracing
- [ ] View results in web dashboard
- [ ] Analyze trace data with pandas

### ✅ Phase 4: SDK Integration (20 minutes)
- [ ] **JavaScript**: Run cell 6 if using Node.js/TypeScript
- [ ] **Python**: Run cell 7 for Python integration
- [ ] Test with real AI application
- [ ] Monitor traces in real-time

## Summary: SDK Availability

### 🟢 **Available Now:**
- **JavaScript/TypeScript SDK** - Full featured, production ready
- **Web Interface** - Complete debugging environment  
- **VS Code Extension** - IDE integration

### 🟡 **Available via API:**
- **Python Integration** - Direct HTTP API calls (cell 7)
- **Jupyter Integration** - Custom helper class (cell 8)
- **Any Language** - REST API compatible

### 🔴 **Planned/In Development:**
- **Official Python SDK** - Native package
- **Jupyter Widgets** - Rich notebook integration
- **Additional language SDKs** - Go, Rust, etc.

## Next Steps

1. **Start with Web Interface** - Get familiar with the concepts
2. **Choose your SDK** - JavaScript for full features, API calls for Python
3. **Integrate gradually** - Start with manual traces, add auto-tracing later
4. **Share with team** - Use collaboration features for debugging together

**The key insight: FlowScope makes invisible AI operations visible for debugging!**

# 🧪 Comprehensive Web Interface Test Suite

## Complete Feature Testing for FlowScope Web Interface

This test suite covers **ALL features** of the FlowScope web interface:
- ✅ Session Management (create, update, delete)
- ✅ Trace Operations (add, view, filter) 
- ✅ Visualization Views (Chain, Timeline, Node Details)
- ✅ Collaboration Features (comments, bookmarks, sharing)
- ✅ Real-time Updates (WebSocket connections)
- ✅ Performance Monitoring (trace analytics)
- ✅ User Preferences (themes, layouts, settings)

### Test Categories
1. **Core Functionality** - Sessions, traces, basic operations
2. **Visualization Testing** - All view modes and interactions
3. **Collaboration Features** - Comments, bookmarks, sharing
4. **Advanced Features** - Real-time updates, analytics, preferences

In [ ]:
// 🧪 COMPREHENSIVE WEB INTERFACE TEST SUITE
// Copy and paste this entire code block into browser console at http://localhost:5173

class FlowScopeTestSuite {
    constructor() {
        this.baseUrl = 'http://localhost:3001/api';
        this.testResults = [];
        this.testSessions = [];
        this.testTraces = [];
    }

    // Test result logging
    logTest(testName, passed, details = '') {
        const result = { testName, passed, details, timestamp: new Date() };
        this.testResults.push(result);
        const status = passed ? '✅' : '❌';
        console.log(`${status} ${testName}${details ? ': ' + details : ''}`);
        return passed;
    }

    // 1. CORE FUNCTIONALITY TESTS
    async testSessionManagement() {
        console.log('\n🔄 Testing Session Management...');
        
        try {
            // Test 1: Create Session
            const sessionResponse = await fetch(`${this.baseUrl}/sessions`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    name: 'Test Suite Session',
                    metadata: { 
                        testSuite: true, 
                        created: Date.now(),
                        features: ['core', 'visualization', 'collaboration']
                    }
                })
            });
            
            if (!sessionResponse.ok) throw new Error('Session creation failed');
            const session = await sessionResponse.json();
            this.testSessions.push(session);
            this.logTest('Create Session', true, `ID: ${session.id}`);

            // Test 2: Get Session
            const getResponse = await fetch(`${this.baseUrl}/sessions/${session.id}`);
            if (!getResponse.ok) throw new Error('Get session failed');
            const retrievedSession = await getResponse.json();
            this.logTest('Get Session', retrievedSession.id === session.id);

            // Test 3: Update Session
            const updateResponse = await fetch(`${this.baseUrl}/sessions/${session.id}`, {
                method: 'PUT',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    name: 'Updated Test Suite Session',
                    metadata: { ...session.metadata, updated: true }
                })
            });
            this.logTest('Update Session', updateResponse.ok);

            // Test 4: List All Sessions
            const listResponse = await fetch(`${this.baseUrl}/sessions`);
            const sessions = await listResponse.json();
            this.logTest('List Sessions', Array.isArray(sessions) && sessions.length > 0);

        } catch (error) {
            this.logTest('Session Management', false, error.message);
        }
    }

    async testTraceOperations() {
        console.log('\n🔄 Testing Trace Operations...');
        
        if (this.testSessions.length === 0) {
            this.logTest('Trace Operations', false, 'No test session available');
            return;
        }

        const sessionId = this.testSessions[0].id;

        try {
            // Test 1: Add Single Trace
            const traceTypes = ['prompt', 'function_call', 'response', 'error', 'agent_step'];
            
            for (let i = 0; i < traceTypes.length; i++) {
                const traceType = traceTypes[i];
                const traceResponse = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        sessionId,
                        chainId: `test-chain-${i + 1}`,
                        timestamp: Date.now() + i * 1000,
                        type: traceType,
                        status: i % 2 === 0 ? 'completed' : 'failed',
                        data: {
                            testType: traceType,
                            step: i + 1,
                            prompt: `Test ${traceType} prompt`,
                            response: `Test ${traceType} response`,
                            duration: Math.random() * 2000,
                            tokens: { input: 10 + i, output: 15 + i, total: 25 + (i * 2) },
                            cost: (Math.random() * 0.01).toFixed(4)
                        }
                    })
                });

                if (traceResponse.ok) {
                    const trace = await traceResponse.json();
                    this.testTraces.push(trace);
                    this.logTest(`Add ${traceType} Trace`, true, `ID: ${trace.id}`);
                } else {
                    this.logTest(`Add ${traceType} Trace`, false);
                }
            }

            // Test 2: Bulk Add Traces
            const bulkTraces = Array.from({ length: 3 }, (_, i) => ({
                sessionId,
                chainId: `bulk-chain-${i + 1}`,
                timestamp: Date.now() + (i + 10) * 1000,
                type: 'prompt',
                status: 'completed',
                data: {
                    bulkTest: true,
                    batchNumber: i + 1,
                    prompt: `Bulk test prompt ${i + 1}`,
                    response: `Bulk test response ${i + 1}`
                }
            }));

            const bulkResponse = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces/bulk`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify(bulkTraces)
            });

            this.logTest('Bulk Add Traces', bulkResponse.ok, `Added ${bulkTraces.length} traces`);

            // Test 3: Get Session Traces
            const tracesResponse = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces`);
            const traces = await tracesResponse.json();
            this.logTest('Get Session Traces', Array.isArray(traces) && traces.length >= 5, `Found ${traces.length} traces`);

        } catch (error) {
            this.logTest('Trace Operations', false, error.message);
        }
    }

    async testVisualizationFeatures() {
        console.log('\n🔄 Testing Visualization Features...');
        
        // Test visualization by checking DOM elements (requires the web interface to be loaded)
        try {
            // Test 1: Check if main visualization container exists
            const chainView = document.querySelector('[data-testid="chain-visualization"]') || 
                             document.querySelector('.chain-view') ||
                             document.querySelector('#chain-view');
            this.logTest('Chain Visualization Component', !!chainView);

            // Test 2: Check for timeline view
            const timelineView = document.querySelector('[data-testid="timeline-view"]') || 
                                document.querySelector('.timeline-view') ||
                                document.querySelector('#timeline-view');
            this.logTest('Timeline View Component', !!timelineView);

            // Test 3: Check for node details panel
            const nodeDetails = document.querySelector('[data-testid="node-details"]') || 
                               document.querySelector('.node-details') ||
                               document.querySelector('#node-details');
            this.logTest('Node Details Component', !!nodeDetails);

            // Test 4: Check for trace inspector
            const traceInspector = document.querySelector('[data-testid="trace-inspector"]') || 
                                  document.querySelector('.trace-inspector') ||
                                  document.querySelector('#trace-inspector');
            this.logTest('Trace Inspector Component', !!traceInspector);

            // Test 5: Check for session selector
            const sessionSelector = document.querySelector('select') || 
                                   document.querySelector('.session-selector') ||
                                   document.querySelector('[data-testid="session-selector"]');
            this.logTest('Session Selector', !!sessionSelector);

        } catch (error) {
            this.logTest('Visualization Features', false, error.message);
        }
    }

    async testCollaborationFeatures() {
        console.log('\n🔄 Testing Collaboration Features...');
        
        if (this.testSessions.length === 0 || this.testTraces.length === 0) {
            this.logTest('Collaboration Features', false, 'No test data available');
            return;
        }

        try {
            const sessionId = this.testSessions[0].id;
            const traceId = this.testTraces[0]?.id;

            // Test 1: Add Comment
            if (traceId) {
                const commentResponse = await fetch(`${this.baseUrl}/collaboration/comments`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        traceId,
                        sessionId,
                        userId: 'test-user',
                        content: 'Test comment for trace analysis',
                        type: 'question'
                    })
                });
                this.logTest('Add Comment', commentResponse.ok);
            }

            // Test 2: Add Bookmark
            if (traceId) {
                const bookmarkResponse = await fetch(`${this.baseUrl}/bookmarks`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        userId: 'test-user',
                        traceId,
                        sessionId,
                        title: 'Important trace for review',
                        description: 'This trace shows interesting behavior',
                        tags: ['important', 'review', 'test']
                    })
                });
                this.logTest('Add Bookmark', bookmarkResponse.ok);
            }

            // Test 3: Create Share Link
            const shareResponse = await fetch(`${this.baseUrl}/sharing/links`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    sessionId,
                    shareType: 'public',
                    expiresAt: Date.now() + (24 * 60 * 60 * 1000), // 24 hours
                    allowComments: true,
                    allowDownload: false
                })
            });
            this.logTest('Create Share Link', shareResponse.ok);

        } catch (error) {
            this.logTest('Collaboration Features', false, error.message);
        }
    }

    async testAdvancedFeatures() {
        console.log('\n🔄 Testing Advanced Features...');
        
        try {
            // Test 1: Health Check
            const healthResponse = await fetch(`${this.baseUrl}/health`);
            this.logTest('Backend Health Check', healthResponse.ok);

            // Test 2: Database Debug Info
            const debugResponse = await fetch(`${this.baseUrl}/sessions/debug/database-sessions`);
            if (debugResponse.ok) {
                const debugInfo = await debugResponse.json();
                this.logTest('Database Debug Info', typeof debugInfo.total === 'number');
            }

            // Test 3: User Preferences
            const prefsResponse = await fetch(`${this.baseUrl}/user-preferences/test-user`);
            this.logTest('User Preferences Endpoint', prefsResponse.status === 200 || prefsResponse.status === 404);

            // Test 4: WebSocket Connection (if available)
            try {
                const ws = new WebSocket('ws://localhost:3001');
                ws.onopen = () => {
                    this.logTest('WebSocket Connection', true);
                    ws.close();
                };
                ws.onerror = () => {
                    this.logTest('WebSocket Connection', false, 'Connection failed');
                };
            } catch (wsError) {
                this.logTest('WebSocket Connection', false, 'WebSocket not available');
            }

        } catch (error) {
            this.logTest('Advanced Features', false, error.message);
        }
    }

    async testUserInterfaceElements() {
        console.log('\n🔄 Testing UI Elements...');
        
        try {
            // Test critical UI elements
            const tests = [
                { name: 'Main Container', selector: 'body, #root, .app' },
                { name: 'Navigation Menu', selector: 'nav, .navigation, .menu' },
                { name: 'Session Panel', selector: '.session-panel, .sessions, [data-testid*="session"]' },
                { name: 'Visualization Area', selector: '.visualization, .chart-container, canvas, svg' },
                { name: 'Control Buttons', selector: 'button, .btn, [role="button"]' },
                { name: 'Loading Indicators', selector: '.loader, .loading, .spinner' },
                { name: 'Error Boundaries', selector: '.error, .error-boundary' },
                { name: 'Responsive Layout', selector: '.container, .wrapper, .layout' }
            ];

            tests.forEach(test => {
                const element = document.querySelector(test.selector);
                this.logTest(`UI Element: ${test.name}`, !!element);
            });

            // Test responsive design
            const viewport = {
                width: window.innerWidth,
                height: window.innerHeight
            };
            this.logTest('Viewport Size', viewport.width > 0 && viewport.height > 0, `${viewport.width}x${viewport.height}`);

        } catch (error) {
            this.logTest('UI Elements', false, error.message);
        }
    }

    // Cleanup test data
    async cleanup() {
        console.log('\n🧹 Cleaning up test data...');
        
        try {
            // Delete test sessions (which will cascade delete traces)
            for (const session of this.testSessions) {
                const deleteResponse = await fetch(`${this.baseUrl}/sessions/${session.id}`, {
                    method: 'DELETE'
                });
                this.logTest(`Cleanup Session ${session.id}`, deleteResponse.ok);
            }
        } catch (error) {
            this.logTest('Cleanup', false, error.message);
        }
    }

    // Generate test report
    generateReport() {
        console.log('\n📊 TEST SUITE RESULTS');
        console.log('========================');
        
        const total = this.testResults.length;
        const passed = this.testResults.filter(r => r.passed).length;
        const failed = total - passed;
        
        console.log(`Total Tests: ${total}`);
        console.log(`✅ Passed: ${passed}`);
        console.log(`❌ Failed: ${failed}`);
        console.log(`Success Rate: ${((passed/total) * 100).toFixed(1)}%`);
        
        if (failed > 0) {
            console.log('\n❌ Failed Tests:');
            this.testResults.filter(r => !r.passed).forEach(test => {
                console.log(`  - ${test.testName}: ${test.details}`);
            });
        }

        console.log('\n📋 Next Steps:');
        console.log('1. Open http://localhost:5173 to view created test data');
        console.log('2. Test manual UI interactions (clicking, dragging, etc.)');
        console.log('3. Test visualization updates and real-time features');
        console.log('4. Run cleanup() when finished testing');
        
        return { total, passed, failed, successRate: (passed/total) * 100 };
    }

    // Main test runner
    async runAllTests() {
        console.log('🚀 Starting FlowScope Web Interface Test Suite...');
        console.log('================================================');
        
        try {
            await this.testSessionManagement();
            await this.testTraceOperations();
            await this.testVisualizationFeatures();
            await this.testCollaborationFeatures();
            await this.testAdvancedFeatures();
            await this.testUserInterfaceElements();
            
            return this.generateReport();
            
        } catch (error) {
            console.error('❌ Test suite failed:', error);
            return this.generateReport();
        }
    }
}

// 🏃‍♂️ RUN THE TEST SUITE
// Uncomment the lines below to run all tests:

// const testSuite = new FlowScopeTestSuite();
// testSuite.runAllTests().then(results => {
//     console.log('🎉 Test suite completed!', results);
// });

// Or run individual test categories:
// testSuite.testSessionManagement();
// testSuite.testTraceOperations();
// testSuite.testVisualizationFeatures();

console.log('📝 Test suite loaded! Run testSuite.runAllTests() to start testing.');

In [ ]:
// 🎯 PERFORMANCE & REAL-TIME TESTING EXTENSIONS

// Add these methods to the FlowScopeTestSuite class above

// Performance Testing
FlowScopeTestSuite.prototype.testPerformance = async function() {
    console.log('\n⚡ Testing Performance...');
    
    try {
        // Test 1: Large Session Creation
        const startTime = Date.now();
        const largeSessionResponse = await fetch(`${this.baseUrl}/sessions`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                name: 'Performance Test Session',
                metadata: { 
                    performance: true,
                    traces: 1000,
                    complexity: 'high'
                }
            })
        });
        
        if (largeSessionResponse.ok) {
            const session = await largeSessionResponse.json();
            const sessionTime = Date.now() - startTime;
            this.logTest('Session Creation Performance', sessionTime < 1000, `${sessionTime}ms`);
            this.testSessions.push(session);
        }

        // Test 2: Bulk Trace Performance
        const sessionId = this.testSessions[0]?.id;
        if (sessionId) {
            const bulkTraces = Array.from({ length: 100 }, (_, i) => ({
                sessionId,
                chainId: `perf-chain-${i}`,
                timestamp: Date.now() + i * 10,
                type: 'prompt',
                status: 'completed',
                data: {
                    performanceTest: true,
                    iteration: i,
                    prompt: `Performance test prompt ${i}`,
                    response: `Performance test response ${i}`,
                    tokens: { input: 50, output: 75, total: 125 }
                }
            }));

            const bulkStartTime = Date.now();
            const bulkResponse = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces/bulk`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify(bulkTraces)
            });
            
            const bulkTime = Date.now() - bulkStartTime;
            this.logTest('Bulk Trace Performance', bulkResponse.ok && bulkTime < 5000, `${bulkTime}ms for 100 traces`);
        }

        // Test 3: Memory Usage (if available)
        if (window.performance && window.performance.memory) {
            const memory = window.performance.memory;
            this.logTest('Memory Usage Check', memory.usedJSHeapSize < memory.jsHeapSizeLimit, 
                `Used: ${(memory.usedJSHeapSize / 1024 / 1024).toFixed(1)}MB`);
        }

        // Test 4: Page Load Performance
        if (window.performance && window.performance.timing) {
            const timing = window.performance.timing;
            const loadTime = timing.loadEventEnd - timing.navigationStart;
            this.logTest('Page Load Performance', loadTime < 5000, `${loadTime}ms`);
        }

    } catch (error) {
        this.logTest('Performance Testing', false, error.message);
    }
};

// Real-time Features Testing
FlowScopeTestSuite.prototype.testRealTimeFeatures = async function() {
    console.log('\n🔄 Testing Real-time Features...');
    
    try {
        // Test 1: Live Trace Updates
        const sessionId = this.testSessions[0]?.id;
        if (sessionId) {
            // Simulate real-time trace additions
            const realTimeTraces = Array.from({ length: 5 }, (_, i) => ({
                sessionId,
                chainId: `realtime-chain-${i}`,
                timestamp: Date.now() + (i * 2000),
                type: 'agent_step',
                status: 'running',
                data: {
                    realTime: true,
                    step: i + 1,
                    action: `Real-time action ${i + 1}`,
                    progress: (i + 1) * 20
                }
            }));

            let successCount = 0;
            for (const trace of realTimeTraces) {
                const response = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(trace)
                });
                
                if (response.ok) {
                    successCount++;
                    // Wait a bit to simulate real-time updates
                    await new Promise(resolve => setTimeout(resolve, 500));
                }
            }
            
            this.logTest('Real-time Trace Updates', successCount === realTimeTraces.length, 
                `${successCount}/${realTimeTraces.length} traces added`);
        }

        // Test 2: WebSocket Events (if WebSocket is available)
        if (typeof WebSocket !== 'undefined') {
            const wsTest = new Promise((resolve) => {
                try {
                    const ws = new WebSocket('ws://localhost:3001');
                    let eventReceived = false;
                    
                    ws.onopen = () => {
                        // Send test event
                        ws.send(JSON.stringify({
                            type: 'test_event',
                            data: { testSuite: true, timestamp: Date.now() }
                        }));
                    };
                    
                    ws.onmessage = (event) => {
                        eventReceived = true;
                        ws.close();
                        resolve(true);
                    };
                    
                    ws.onerror = () => {
                        resolve(false);
                    };
                    
                    // Timeout after 3 seconds
                    setTimeout(() => {
                        if (!eventReceived) {
                            ws.close();
                            resolve(false);
                        }
                    }, 3000);
                    
                } catch (error) {
                    resolve(false);
                }
            });
            
            const wsResult = await wsTest;
            this.logTest('WebSocket Communication', wsResult);
        }

        // Test 3: Auto-refresh/Polling
        const refreshTest = await fetch(`${this.baseUrl}/sessions`);
        if (refreshTest.ok) {
            const sessions1 = await refreshTest.json();
            
            // Wait and fetch again
            await new Promise(resolve => setTimeout(resolve, 1000));
            const refreshTest2 = await fetch(`${this.baseUrl}/sessions`);
            const sessions2 = await refreshTest2.json();
            
            this.logTest('Auto-refresh Capability', refreshTest2.ok, 
                `Sessions count: ${sessions1.length} → ${sessions2.length}`);
        }

    } catch (error) {
        this.logTest('Real-time Features', false, error.message);
    }
};

// Data Validation Testing
FlowScopeTestSuite.prototype.testDataValidation = async function() {
    console.log('\n🔍 Testing Data Validation...');
    
    try {
        // Test 1: Invalid Session Data
        const invalidSessionResponse = await fetch(`${this.baseUrl}/sessions`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                // Missing required fields
                metadata: { invalid: true }
            })
        });
        
        this.logTest('Invalid Session Rejection', !invalidSessionResponse.ok || invalidSessionResponse.status >= 400);

        // Test 2: Invalid Trace Data
        const sessionId = this.testSessions[0]?.id;
        if (sessionId) {
            const invalidTraceResponse = await fetch(`${this.baseUrl}/sessions/${sessionId}/traces`, {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({
                    // Missing required fields
                    invalidTrace: true
                })
            });
            
            this.logTest('Invalid Trace Rejection', !invalidTraceResponse.ok || invalidTraceResponse.status >= 400);
        }

        // Test 3: SQL Injection Protection
        const maliciousSessionResponse = await fetch(`${this.baseUrl}/sessions`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({
                name: "'; DROP TABLE sessions; --",
                metadata: { malicious: true }
            })
        });
        
        // Should either reject or sanitize the input
        this.logTest('SQL Injection Protection', maliciousSessionResponse.status !== 500);

        // Test 4: Large Payload Handling
        const largePayload = {
            name: 'Large Payload Test',
            metadata: {
                largeData: 'x'.repeat(10000) // 10KB string
            }
        };
        
        const largePayloadResponse = await fetch(`${this.baseUrl}/sessions`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify(largePayload)
        });
        
        this.logTest('Large Payload Handling', largePayloadResponse.status < 500);

    } catch (error) {
        this.logTest('Data Validation', false, error.message);
    }
};

// Extended test runner with new tests
FlowScopeTestSuite.prototype.runExtendedTests = async function() {
    console.log('🚀 Starting Extended FlowScope Test Suite...');
    console.log('===============================================');
    
    try {
        // Run core tests first
        await this.testSessionManagement();
        await this.testTraceOperations();
        await this.testVisualizationFeatures();
        await this.testCollaborationFeatures();
        await this.testAdvancedFeatures();
        await this.testUserInterfaceElements();
        
        // Run extended tests
        await this.testPerformance();
        await this.testRealTimeFeatures();
        await this.testDataValidation();
        
        return this.generateReport();
        
    } catch (error) {
        console.error('❌ Extended test suite failed:', error);
        return this.generateReport();
    }
};

console.log('⚡ Performance and real-time testing extensions loaded!');
console.log('Run testSuite.runExtendedTests() for comprehensive testing.');

## 📋 Manual Testing Checklist

After running the automated test suite, perform these manual tests in the web interface:

### 🎯 Core User Workflows
- [ ] **Session Creation**: Create a new session and verify it appears in the session list
- [ ] **Session Selection**: Switch between different sessions and verify data loads correctly
- [ ] **Trace Visualization**: View trace chains in the visualization area
- [ ] **Node Interaction**: Click on individual trace nodes to view details
- [ ] **Timeline Navigation**: Use timeline controls to navigate through traces
- [ ] **Zoom & Pan**: Test zooming in/out and panning around the visualization

### 🖥️ User Interface Testing
- [ ] **Responsive Design**: Resize browser window and verify layout adapts
- [ ] **Dark/Light Theme**: Toggle between themes if available
- [ ] **Sidebar Controls**: Test all buttons and controls in the sidebar
- [ ] **Context Menus**: Right-click on elements to test context menus
- [ ] **Keyboard Shortcuts**: Test any available keyboard shortcuts
- [ ] **Loading States**: Verify loading indicators appear during data fetching

### 🔄 Data Operations
- [ ] **Real-time Updates**: Verify new traces appear automatically if live mode is enabled
- [ ] **Data Refresh**: Use refresh/reload buttons to update data
- [ ] **Search/Filter**: Test any search or filtering functionality
- [ ] **Export/Download**: Test data export features if available
- [ ] **Error Handling**: Test with invalid data or network errors

### 🤝 Collaboration Features
- [ ] **Comments**: Add, edit, and delete comments on traces
- [ ] **Bookmarks**: Create and manage bookmarks for important traces
- [ ] **Sharing**: Generate and test share links
- [ ] **Multi-user**: Test with multiple browser tabs/users if supported

### 🔍 Advanced Features
- [ ] **Performance**: Test with large datasets (100+ traces)
- [ ] **Memory Usage**: Monitor browser memory usage during extended use
- [ ] **Browser Compatibility**: Test in Chrome, Firefox, Safari, and Edge
- [ ] **Mobile/Tablet**: Test on mobile devices if responsive design is implemented

---

## 🚀 Quick Start Testing Guide

### Step 1: Start the Application
```bash
# Make sure both backend and frontend are running
cd FlowScope
npm run dev  # or your start command
```

### Step 2: Run Automated Tests
1. Open browser to `http://localhost:5173`
2. Open Developer Console (F12)
3. Copy and paste the test suite code from the cell above
4. Run the tests:
```javascript
const testSuite = new FlowScopeTestSuite();
testSuite.runExtendedTests().then(results => {
    console.log('🎉 All tests completed!', results);
});
```

### Step 3: Manual Testing
1. Use the manual checklist above
2. Interact with the created test data
3. Test edge cases and error scenarios
4. Document any issues found

### Step 4: Cleanup
```javascript
// Clean up test data when finished
testSuite.cleanup();
```

---

## 🔧 Troubleshooting Common Issues

### Backend Not Responding
- Verify backend is running on `http://localhost:3001`
- Check backend logs for errors
- Ensure database is properly initialized

### CORS Errors
- Check if frontend and backend are running on correct ports
- Verify CORS configuration in backend

### Test Failures
- Check browser console for detailed error messages
- Verify API endpoints exist and are working
- Check network tab for failed requests

### UI Issues
- Clear browser cache and reload
- Check for JavaScript errors in console
- Verify all required dependencies are installed

---

## 📊 Expected Test Results

A successful test run should show:
- **95%+ Success Rate** for core functionality
- **All UI Elements** present and functional
- **Performance Tests** completing within time limits
- **No Critical Errors** in browser console

If tests fail, check the specific error messages and troubleshoot accordingly.

In [ ]:
// 🧹 DATABASE CLEANUP UTILITY
// Run this to clear ALL test data from the FlowScope database

async function clearAllTestData() {
    console.log('🧹 Starting database cleanup...');
    
    try {
        // Get all sessions
        const response = await fetch('http://localhost:3001/api/sessions');
        if (!response.ok) {
            throw new Error(`Failed to fetch sessions: ${response.status}`);
        }
        
        const sessions = await response.json();
        console.log(`📋 Found ${sessions.length} sessions to delete`);
        
        if (sessions.length === 0) {
            console.log('✅ Database is already clean!');
            return;
        }
        
        // Delete each session
        let deletedCount = 0;
        for (const session of sessions) {
            try {
                console.log(`🗑️ Deleting: ${session.name} (${session.id})`);
                const deleteResponse = await fetch(`http://localhost:3001/api/sessions/${session.id}`, {
                    method: 'DELETE'
                });
                
                if (deleteResponse.ok) {
                    deletedCount++;
                    console.log(`✅ Deleted: ${session.id}`);
                } else {
                    console.log(`❌ Failed to delete: ${session.id}`);
                }
                
                // Small delay to avoid overwhelming server
                await new Promise(resolve => setTimeout(resolve, 100));
                
            } catch (error) {
                console.log(`❌ Error deleting ${session.id}: ${error.message}`);
            }
        }
        
        // Verify cleanup
        const verifyResponse = await fetch('http://localhost:3001/api/sessions');
        const remainingSessions = await verifyResponse.json();
        
        console.log('\n📊 CLEANUP SUMMARY');
        console.log('==================');
        console.log(`✅ Sessions deleted: ${deletedCount}`);
        console.log(`📋 Remaining sessions: ${remainingSessions.length}`);
        
        if (remainingSessions.length === 0) {
            console.log('🎉 Database successfully cleared!');
        } else {
            console.log('⚠️ Some sessions remain:');
            remainingSessions.forEach(s => console.log(`  - ${s.name}`));
        }
        
    } catch (error) {
        console.error('❌ Cleanup failed:', error);
    }
}

// 🎯 IMPROVED VISUALIZATION TESTING
// Updated selectors to match actual FlowScope UI components

function testActualVisualizationComponents() {
    console.log('🔄 Testing Actual FlowScope UI Components...');
    
    // More comprehensive DOM element detection
    const componentTests = [
        {
            name: 'React Root Container',
            selectors: ['#root', '#app', '.app', '[data-reactroot]'],
            required: true
        },
        {
            name: 'Main Application Container',
            selectors: ['.container', '.main-container', '.app-container', 'main'],
            required: true
        },
        {
            name: 'Visualization Canvas/SVG',
            selectors: ['canvas', 'svg', '.visualization', '.chart', '.graph'],
            required: false
        },
        {
            name: 'Chain/Flow View',
            selectors: [
                '.chain-view', '.flow-view', '.node-graph', 
                '[class*="chain"]', '[class*="flow"]', '[class*="graph"]'
            ],
            required: false
        },
        {
            name: 'Timeline Component',
            selectors: [
                '.timeline', '.timeline-view', '.time-axis',
                '[class*="timeline"]', '[class*="time"]'
            ],
            required: false
        },
        {
            name: 'Session Controls',
            selectors: [
                'select', '.session-selector', '.dropdown',
                '[class*="session"]', '[class*="selector"]'
            ],
            required: false
        },
        {
            name: 'Sidebar/Panel',
            selectors: [
                '.sidebar', '.panel', '.side-panel', 
                '[class*="sidebar"]', '[class*="panel"]'
            ],
            required: false
        },
        {
            name: 'Navigation/Menu',
            selectors: ['nav', '.nav', '.navigation', '.menu', '.navbar'],
            required: false
        }
    ];
    
    const results = [];
    
    componentTests.forEach(test => {
        let found = false;
        let foundElement = null;
        
        for (const selector of test.selectors) {
            const element = document.querySelector(selector);
            if (element) {
                found = true;
                foundElement = element;
                break;
            }
        }
        
        const status = found ? '✅' : (test.required ? '❌' : '⚠️');
        console.log(`${status} ${test.name}${found ? ` (found: ${foundElement.tagName.toLowerCase()})` : ''}`);
        
        results.push({
            name: test.name,
            found,
            required: test.required,
            element: foundElement
        });
    });
    
    // Additional DOM inspection
    console.log('\n🔍 DOM Structure Analysis:');
    console.log(`- Document title: "${document.title}"`);
    console.log(`- Body classes: ${document.body.className || 'none'}`);
    console.log(`- Total elements: ${document.querySelectorAll('*').length}`);
    console.log(`- React elements: ${document.querySelectorAll('[data-reactroot], [data-react-*]').length}`);
    
    // Check for common React/TypeScript patterns
    const reactPatterns = [
        '.jsx-', '.tsx-', '[class*="jsx"]', '[class*="tsx"]',
        '[class*="react"]', '[class*="component"]'
    ];
    
    reactPatterns.forEach(pattern => {
        const elements = document.querySelectorAll(pattern);
        if (elements.length > 0) {
            console.log(`- Found ${elements.length} elements matching ${pattern}`);
        }
    });
    
    return results;
}

// Run database cleanup
console.log('🚀 Database cleanup utility loaded!');
console.log('📝 Usage:');
console.log('  clearAllTestData() - Remove all sessions from database');
console.log('  testActualVisualizationComponents() - Inspect actual UI components');
console.log('');
console.log('💡 Run clearAllTestData() to clean the database now!');